In [2]:
import os
from dotenv import load_dotenv
load_dotenv() #load all the environment variables

True

In [3]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [ ]:
##Embedding techniques
## Converting text into vectors

In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002556BB9FD60>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002556A13E470>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
text="This is a demo project on OPENAI embedding"
query_result = embeddings.embed_query(text)
query_result

ConnectionError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002556D07F7F0>: Failed to resolve 'openaipublic.blob.core.windows.net' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
len(query_result) #gives dimensions

In [ ]:
#to change dimensions of embeddings to 1024 instead of 3072 
embeddings_1024 = OpenAIEmbeddings(model="text-embedding-3-large",dimensions=1024)
text="This is a demo project on OPENAI embedding"
query_result_1024 = embeddings_1024.embed_query(text)
len(query_result_1024)


In [6]:
## text loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader('speech.txt') 
#text documents will be made when we call load function
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'speech.txt'}, page_content='Making the World “Safe for Democracy”: Woodrow Wilson Asks for War\nOn April 2, 1917, President Woodrow Wilson went before a joint session of Congress to seek a Declaration of War against Germany in order that the world “be made safe for democracy.” Four days later, Congress voted to declare war, with six senators and fifty House members dissenting. “It is a fearful thing,” he told Congress in his speech, “to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance.” Wilson did not exaggerate; in 1917 the war in Europe had already lasted two-and-a-half bloody years and had become one of the most murderous conflicts in human history. By the time the war ended a year and a half later, an entire generation was decimated—France alone lost half its men between the ages of twenty and thirty-two. The maimed bodies of millions of European men who survi

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
#splitting speech
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
final_documents = text_splitter.split_documents(text_documents) #if speech as a list is not passed our splitting will be character by character
print(final_documents)

[Document(metadata={'source': 'speech.txt'}, page_content='Making the World “Safe for Democracy”: Woodrow Wilson Asks for War'), Document(metadata={'source': 'speech.txt'}, page_content='On April 2, 1917, President Woodrow Wilson went before a joint session of Congress to seek a Declaration of War against Germany in order that the world “be made safe for democracy.” Four days later, Congress voted to declare war, with six senators and fifty House members dissenting. “It is a fearful thing,” he told Congress in his speech, “to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance.” Wilson did'), Document(metadata={'source': 'speech.txt'}, page_content='itself seeming to be in the balance.” Wilson did not exaggerate; in 1917 the war in Europe had already lasted two-and-a-half bloody years and had become one of the most murderous conflicts in human history. By the time the war ended a year and a half l

In [ ]:
#convert these chunk of documents into vectors, then store those vectors into vector DB


In [ ]:
##since we have list of documents so we combine vector embedding and VectorStoreDB - Chroma #open source db
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(final_documents,embeddings_1024) #parameters- our  list of documents and embedding technique
db

In [ ]:
#how to query from this vector db store
query="It will be all the easier for us to conduct ourselves as belligerents"

#use this db and do some similarity search
retrieved_results = db.similarity_search(query)
print(retrieved_results)